d ##### Usecase:Get sales data by orders
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data per day
###### Problem statement-2:Get sales data per month
###### Problem statement-3:Get sales data per year
###### Problem statement-4:Get monthly sales data from third quarter of 2013 and first quarter of 2014 when Max sales >100 
###### Problem statement-5:Get monthly sales data from last quarter of 2013 where soldquantity_permonth > 25000 
###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where averagesale > 200 and Numberofsales >=600
###### Problem statement-7:Get daily sales data  where soldquantity_perday > 1500
###### Problem statement-8:Get final qurater of 2013 sales data of TOP5 revenue generating order for each month
###### Problem statement-9:Get average revenue per day and all the orders which are more than average.
###### Problem statement-10:Percentage of order items in Order revenue

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,round,avg,rank, col
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.functions import unix_timestamp, lit
spark = SparkSession.builder.master('local').appName('SalesDataForeachorder').enableHiveSupport().getOrCreate()

In [3]:
ordersdf = spark.read.csv("/FileStore/tables/retaildbtext/Orders.txt", sep=',',schema ='order_id int,order_date string,order_customer_id int, order_status string')
ordersdf.show(5,False)

+--------+----------+-----------------+---------------+
order_id|order_date|order_customer_id|order_status |
+--------+----------+-----------------+---------------+
1 |2013-07-25|11599 |CLOSED |
2 |2013-07-25|256 |PENDING_PAYMENT|
3 |2013-07-25|12111 |COMPLETE |
4 |2013-07-25|8827 |CLOSED |
5 |2013-07-25|11318 |COMPLETE |
+--------+----------+-----------------+---------------+
only showing top 5 rows

In [4]:
orderItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Order_Items.txt", sep=',', schema='order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float')
orderItemsdf.show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [5]:
ordersMap = ordersdf.select('order_id','order_date')
ordersMap.show(5,False)


+--------+----------+
order_id|order_date|
+--------+----------+
1 |2013-07-25|
2 |2013-07-25|
3 |2013-07-25|
4 |2013-07-25|
5 |2013-07-25|
+--------+----------+
only showing top 5 rows

In [6]:
orderItemsMap=orderItemsdf.select('order_item_order_id','order_item_subtotal','order_item_quantity')
orderItemsMap.show(5,False)

+-------------------+-------------------+-------------------+
order_item_order_id|order_item_subtotal|order_item_quantity|
+-------------------+-------------------+-------------------+
1 |299.98 |1 |
2 |199.99 |1 |
2 |250.0 |5 |
2 |129.99 |1 |
4 |49.98 |2 |
+-------------------+-------------------+-------------------+
only showing top 5 rows

In [7]:
ordersdfjoin = ordersMap.join(orderItemsMap, ordersMap.order_id == orderItemsMap.order_item_order_id,'inner')
ordersdfjoin.show(5,False)

+--------+----------+-------------------+-------------------+-------------------+
order_id|order_date|order_item_order_id|order_item_subtotal|order_item_quantity|
+--------+----------+-------------------+-------------------+-------------------+
1 |2013-07-25|1 |299.98 |1 |
2 |2013-07-25|2 |199.99 |1 |
2 |2013-07-25|2 |250.0 |5 |
2 |2013-07-25|2 |129.99 |1 |
4 |2013-07-25|4 |49.98 |2 |
+--------+----------+-------------------+-------------------+-------------------+
only showing top 5 rows

In [8]:
ordersdfjoin.printSchema()

root
-- order_id: integer (nullable = true)
-- order_date: string (nullable = true)
-- order_item_order_id: integer (nullable = true)
-- order_item_subtotal: float (nullable = true)
-- order_item_quantity: integer (nullable = true)

In [9]:
ordersdfSalesmap = ordersdfjoin.select('order_id','order_date','order_item_subtotal','order_item_quantity')
ordersdfSalesmap.show(5,False)

+--------+----------+-------------------+-------------------+
order_id|order_date|order_item_subtotal|order_item_quantity|
+--------+----------+-------------------+-------------------+
1 |2013-07-25|299.98 |1 |
2 |2013-07-25|199.99 |1 |
2 |2013-07-25|250.0 |5 |
2 |2013-07-25|129.99 |1 |
4 |2013-07-25|49.98 |2 |
+--------+----------+-------------------+-------------------+
only showing top 5 rows

###### Problem statement-1:Get sales data per day

In [11]:
salesDatePerDay = ordersdfSalesmap.groupBy('order_date').agg(round(avg('order_item_subtotal'),2).alias("avgsales_perday"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales_perday"),
                                                                count('order_item_subtotal').alias("Numberofsales_perday")).sort('order_date')
salesDatePerDay.coalesce(1).write.csv("/FileStore/tables/salesDatePerDay", compression="none", header ='true')
salesDatePerDay.show(10,False)

+----------+---------------+---------+---------+-----------------+--------------------+
order_date|avgsales_perday|min_sales|max_sales|Totalsales_perday|Numberofsales_perday|
+----------+---------------+---------+---------+-----------------+--------------------+
2013-07-25|201.04 |19.98 |599.99 |68153.83 |339 |
2013-07-26|196.71 |9.99 |499.95 |136520.17 |694 |
2013-07-27|200.94 |17.99 |499.95 |101074.34 |503 |
2013-07-28|198.91 |17.99 |499.95 |87123.08 |438 |
2013-07-29|206.14 |17.99 |499.95 |137287.09 |666 |
2013-07-30|190.27 |25.0 |499.95 |102745.62 |540 |
2013-07-31|205.74 |19.99 |499.95 |131878.06 |641 |
2013-08-01|202.83 |15.99 |499.95 |129001.62 |636 |
2013-08-02|195.96 |19.98 |499.95 |109347.0 |558 |
2013-08-03|196.43 |24.99 |499.95 |95266.89 |485 |
+----------+---------------+---------+---------+-----------------+--------------------+
only showing top 10 rows

###### Problem statement-2: Get sales data  per month

In [13]:
SalesDataPerMonth = ordersdfSalesmap.groupBy(substring('order_date',1,7).alias('order_month')).agg(round(avg('order_item_subtotal'),2).alias("avg_sales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales_permonth"),
                                                                count('order_item_subtotal').alias("Numberofsales_permonth")).sort('order_month')
SalesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonth", compression="none", header ='true')
SalesDataPerMonth.show(20,False)

+-----------+---------+---------+---------+-------------------+----------------------+
order_month|avg_sales|min_sales|max_sales|Totalsales_permonth|Numberofsales_permonth|
+-----------+---------+---------+---------+-------------------+----------------------+
2013-07 |200.15 |9.99 |599.99 |764782.2 |3821 |
2013-08 |198.25 |9.99 |1999.99 |2828658.75 |14268 |
2013-09 |199.71 |9.99 |1999.99 |2934527.33 |14694 |
2013-10 |198.59 |9.99 |1999.99 |2624600.66 |13216 |
2013-11 |199.22 |9.99 |599.99 |3168656.09 |15905 |
2013-12 |199.13 |9.99 |1999.99 |2932964.33 |14729 |
2014-01 |199.4 |9.99 |1999.99 |2924447.07 |14666 |
2014-02 |198.0 |9.99 |999.99 |2778663.71 |14034 |
2014-03 |199.48 |9.99 |1999.99 |2862492.27 |14350 |
2014-04 |197.84 |9.99 |1999.99 |2807789.85 |14192 |
2014-05 |200.17 |9.99 |1999.99 |2753078.27 |13754 |
2014-06 |203.19 |9.99 |1999.99 |2703463.49 |13305 |
2014-07 |198.73 |9.99 |1999.99 |2238496.56 |11264 |
+-----------+---------+---------+---------+-------------------+----------------------+

###### Problem statement-3: Get sales data  per year

In [15]:
salesPerEachYear = ordersdfSalesmap.groupBy(substring('order_date',1,4).alias('order_year')).agg(round(avg('order_item_subtotal'),2).alias("avg_sales_peryear"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales_peryear"),
                                                                count('order_item_subtotal').alias("Numberofsales_peryear")).sort('order_year')
salesPerEachYear.coalesce(1).write.csv("/FileStore/tables/salesPerEachYear", compression="none", header="true")
salesPerEachYear.show(20,False)

+----------+-----------------+---------+---------+------------------+---------------------+
order_year|avg_sales_peryear|min_sales|max_sales|Totalsales_peryear|Numberofsales_peryear|
+----------+-----------------+---------+---------+------------------+---------------------+
2013 |199.06 |9.99 |1999.99 |1.525418937E7 |76633 |
2014 |199.53 |9.99 |1999.99 |1.906843123E7 |95565 |
+----------+-----------------+---------+---------+------------------+---------------------+

###### Problem statement-4:Get monthly sales data from third quarter of 2013 and first quarter of 2014 when Max sales >100

In [17]:

SalesDataPerMonth=ordersdfSalesmap.where((to_date(ordersdfmonthlySalesmap.order_date) >=lit("2013-07-01")) & 
                                                 (to_date(ordersdfmonthlySalesmap.order_date) <=lit("2013-09-30")) |
                                                 (to_date(ordersdfmonthlySalesmap.order_date) >=lit("2014-01-01")) & 
                                                 (to_date(ordersdfmonthlySalesmap.order_date) <=lit("2014-03-31")))
SalesDataPerMonthquarter = SalesDataPerMonth.groupBy(substring('order_date',1,7).alias('order_month')).agg(round(avg('order_item_subtotal'),2).alias("avg_sales_permonth"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales_permonth"),
                                                                count('order_item_subtotal').alias("Numberofsales_permonth"))                                                  
SalesDataPerMonthquarterfinal= SalesDataPerMonthquarter.filter(SalesDataPerMonthquarter.max_sales>=500.00).sort("order_month")
SalesDataPerMonthquarterfinal.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonthquarterfinal", compression="none", header="true")
SalesDataPerMonthquarterfinal.show(20,False)

+-----------+------------------+---------+---------+-------------------+----------------------+
order_month|avg_sales_permonth|min_sales|max_sales|Totalsales_permonth|Numberofsales_permonth|
+-----------+------------------+---------+---------+-------------------+----------------------+
2013-07 |200.15 |9.99 |599.99 |764782.2 |3821 |
2013-08 |198.25 |9.99 |1999.99 |2828658.75 |14268 |
2013-09 |199.71 |9.99 |1999.99 |2934527.33 |14694 |
2014-01 |199.4 |9.99 |1999.99 |2924447.07 |14666 |
2014-02 |198.0 |9.99 |999.99 |2778663.71 |14034 |
2014-03 |199.48 |9.99 |1999.99 |2862492.27 |14350 |
+-----------+------------------+---------+---------+-------------------+----------------------+

###### Problem statement-5:Get monthly sales data from last quarter of 2013 where soldquantity_permonth>25000

In [19]:
SalesDataPerMonth=ordersdfSalesmap.where((to_date(ordersdfmonthlySalesmap.order_date) >=lit("2013-10-01")) & 
                                                 (to_date(ordersdfmonthlySalesmap.order_date) <=lit("2013-12-31")))
SalesDataPerMonthquant = SalesDataPerMonth.groupBy(substring('order_date',1,7).alias('order_month')).agg(round(avg('order_item_subtotal'),2).alias("avg_sales_permonth"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales_permonth"),
                                                                sum('order_item_quantity').alias("soldquantity_permonth"))                                                  
SalesDataPerMonthquantfinal= SalesDataPerMonthquant.filter(SalesDataPerMonthquant.soldquantity_permonth>=25000).sort("order_month")
SalesDataPerMonthquantfinal.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonthquantfina", compression="none", header="true")
SalesDataPerMonthquantfinal.show(20,False)

+-----------+------------------+---------+---------+-------------------+---------------------+
order_month|avg_sales_permonth|min_sales|max_sales|Totalsales_permonth|soldquantity_permonth|
+-----------+------------------+---------+---------+-------------------+---------------------+
2013-10 |198.59 |9.99 |1999.99 |2624600.66 |28904 |
2013-11 |199.22 |9.99 |599.99 |3168656.09 |34705 |
2013-12 |199.13 |9.99 |1999.99 |2932964.33 |31921 |
+-----------+------------------+---------+---------+-------------------+---------------------+

###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where averagesale > 200 and Numberofsales >=600

In [21]:
SalesDataPerMonth=ordersdfSalesmap.where((to_date(ordersdfmonthlySalesmap.order_date) >=lit("2013-10-01")) & 
                                                 (to_date(ordersdfmonthlySalesmap.order_date) <=lit("2013-12-31")) |
                                                 (to_date(ordersdfmonthlySalesmap.order_date) >=lit("2014-04-01")) & 
                                                 (to_date(ordersdfmonthlySalesmap.order_date) <=lit("2014-07-31")))
SalesDataPerdayavgsales = SalesDataPerMonth.groupBy(substring('order_date',1,7).alias('order_month')).agg(round(avg('order_item_subtotal'),2).alias("avg_sales_permonth"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales_permonth"),
                                                                count('order_item_subtotal').alias("Numberofsales_permonth"))                                                 
SalesDataPerdayavgsalesfinal= SalesDataPerdayavgsales.filter((SalesDataPerdayavgsales.avg_sales_permonth>=100) & (SalesDataPerdayavgsales.Numberofsales_permonth>=600)).sort("order_month")
SalesDataPerMonthquantfinal.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonthquantfina", compression="none", header="true")
SalesDataPerdayavgsalesfinal.show(20,False)

+-----------+------------------+---------+---------+-------------------+----------------------+
order_month|avg_sales_permonth|min_sales|max_sales|Totalsales_permonth|Numberofsales_permonth|
+-----------+------------------+---------+---------+-------------------+----------------------+
2013-10 |198.59 |9.99 |1999.99 |2624600.66 |13216 |
2013-11 |199.22 |9.99 |599.99 |3168656.09 |15905 |
2013-12 |199.13 |9.99 |1999.99 |2932964.33 |14729 |
2014-04 |197.84 |9.99 |1999.99 |2807789.85 |14192 |
2014-05 |200.17 |9.99 |1999.99 |2753078.27 |13754 |
2014-06 |203.19 |9.99 |1999.99 |2703463.49 |13305 |
2014-07 |198.73 |9.99 |1999.99 |2238496.56 |11264 |
+-----------+------------------+---------+---------+-------------------+----------------------+

###### Problem statement-7:Get daily sales data  where soldquantity_perday > 1500

In [23]:
SalesDataPerdaysoldquant = ordersdfSalesmap.groupBy('order_date').agg(round(avg('order_item_subtotal'),2).alias("avg_sales_perday"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales_perday"),
                                                                sum('order_item_quantity').alias("soldquantity_perday"))                                                  
SalesDataPerMonthquantfinal= SalesDataPerdaysoldquant.filter(SalesDataPerdaysoldquant.soldquantity_perday>=1500).sort("order_date")
SalesDataPerMonthquantfinal.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonthquantfina", compression="none", header="true")
SalesDataPerMonthquantfinal.show(20,False)

+----------+----------------+---------+---------+-----------------+-------------------+
order_date|avg_sales_perday|min_sales|max_sales|Totalsales_perday|soldquantity_perday|
+----------+----------------+---------+---------+-----------------+-------------------+
2013-07-26|196.71 |9.99 |499.95 |136520.17 |1511 |
2013-07-29|206.14 |17.99 |499.95 |137287.09 |1518 |
2013-09-14|198.69 |21.99 |499.95 |135308.52 |1518 |
2013-09-25|194.21 |19.98 |499.95 |141775.64 |1609 |
2013-09-27|203.48 |14.99 |499.95 |136537.86 |1509 |
2013-10-04|194.75 |24.99 |499.95 |138079.62 |1591 |
2013-10-13|200.22 |9.99 |499.95 |145561.99 |1574 |
2013-11-03|204.19 |14.99 |499.95 |175599.2 |1822 |
2013-11-05|196.77 |14.99 |499.95 |138528.34 |1622 |
2013-11-14|200.9 |15.99 |499.95 |149271.96 |1700 |
2013-11-24|195.91 |15.99 |499.95 |142234.02 |1526 |
2013-12-26|194.84 |15.99 |999.99 |143012.23 |1567 |
2014-01-05|196.81 |9.99 |499.95 |138550.94 |1575 |
2014-02-01|202.86 |15.99 |499.95 |153561.8 |1660 |
2014-02-19|198.96 |24.99 |499.95 |141857.25 |1513 |
2014-04-27|193.36 |15.99 |499.95 |132838.34 |1524 |
2014-06-19|207.59 |19.99 |499.95 |147183.83 |1574 |
2014-06-27|202.24 |14.99 |499.95 |139952.39 |1590 |
2014-07-15|194.46 |15.99 |499.95 |136902.59 |1501 |
+----------+----------------+---------+---------+-----------------+-------------------+

##### Problem statement-8:Get final qurater of 2013 sales data of TOP5 revenue generating orders for each month

In [25]:
SalesDataPerMonth=ordersdfSalesmap.where((to_date(ordersdfmonthlySalesmap.order_date) >=lit("2013-10-01")) & 
                                                 (to_date(ordersdfmonthlySalesmap.order_date) <=lit("2013-12-31"))) 
top5SalesDataPerMonth= SalesDataPerMonth.groupBy(substring('order_date',1,7).alias('order_month'),'order_id').agg(round(avg('order_item_subtotal'),2).alias("avg_sales_permonth"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales_permonth"),
                                                                count('order_item_subtotal').alias("Numberofsales_permonth"))                                                 

window = Window.partitionBy(top5SalesDataPerMonth['order_month']).orderBy(top5SalesDataPerMonth['Totalsales_permonth'].desc())
top5orderspurchasedeachmonth=top5SalesDataPerMonth.select('order_month','order_id','Totalsales_permonth').withColumn('rank', rank().over(window).alias('rank'))
top5rankedorderseachmonth = top5orderspurchasedeachmonth.filter(col('rank')<=5).sort('order_month','Totalsales_permonth',ascending=[1,0])
top5rankedorderseachmonth.coalesce(1).write.csv("/FileStore/tables/top5rankedorderseachmonth", compression="none", header ='true')          
top5rankedorderseachmonth.show(20,False)


+-----------+--------+-------------------+----+
order_month|order_id|Totalsales_permonth|rank|
+-----------+--------+-------------------+----+
2013-10 |68736 |2259.95 |1 |
2013-10 |14539 |1899.9 |2 |
2013-10 |59652 |1799.81 |3 |
2013-10 |13445 |1699.88 |4 |
2013-10 |14775 |1649.8 |5 |
2013-11 |18735 |1819.86 |1 |
2013-11 |17185 |1699.91 |2 |
2013-11 |19754 |1699.81 |3 |
2013-11 |60794 |1649.84 |4 |
2013-11 |17659 |1649.82 |5 |
2013-12 |68766 |2699.9 |1 |
2013-12 |22030 |2039.8 |2 |
2013-12 |23631 |1829.86 |3 |
2013-12 |61289 |1799.89 |4 |
2013-12 |23425 |1759.9 |5 |
+-----------+--------+-------------------+----+

##### Problem statement-9:Get average revenue per day and all the orders which are more than average.

In [27]:
avgrevenueperday=ordersdfSalesmap.groupBy('order_date','order_id').agg(round(sum('order_item_subtotal'),2).alias('order_revenue')) 
ordersmorethanaverage = avgrevenueperday.withColumn('avg_revenue',avg('order_revenue').over(Window.partitionBy('order_date')))
ordersmorethanaveragefinal = ordersmorethanaverage.filter('order_revenue >= avg_revenue').orderBy('order_date',col('order_revenue').desc())
ordersmorethanaveragefinal.coalesce(1).write.csv("/FileStore/tables/ordersmorethanaveragefinal", compression="none", header ='true')
ordersmorethanaveragefinal.show()

+----------+--------+-------------+-----------------+
order_date|order_id|order_revenue| avg_revenue|
+----------+--------+-------------+-----------------+
2013-07-25| 57779| 1649.8|587.5330172413793|
2013-07-25| 84| 1499.87|587.5330172413793|
2013-07-25| 12| 1299.87|587.5330172413793|
2013-07-25| 73| 1279.65|587.5330172413793|
2013-07-25| 28| 1159.9|587.5330172413793|
2013-07-25| 62| 1149.94|587.5330172413793|
2013-07-25| 57764| 1149.92|587.5330172413793|
2013-07-25| 5| 1129.86|587.5330172413793|
2013-07-25| 57788| 1119.86|587.5330172413793|
2013-07-25| 29| 1109.85|587.5330172413793|
2013-07-25| 99| 1099.93|587.5330172413793|
2013-07-25| 57757| 1099.87|587.5330172413793|
2013-07-25| 57782| 1049.85|587.5330172413793|
2013-07-25| 57763| 1029.9|587.5330172413793|
2013-07-25| 57765| 966.9|587.5330172413793|
2013-07-25| 15| 925.91|587.5330172413793|
2013-07-25| 11| 919.79|587.5330172413793|
2013-07-25| 57759| 909.97|587.5330172413793|
2013-07-25| 101| 899.94|587.5330172413793|
2013-07-25| 63| 899.92|587.5330172413793|
+----------+--------+-------------+-----------------+
only showing top 20 rows

##### Problem statement-10:Percentage of order items in Order revenue

In [29]:
percentageoforderItems = orderItemsdf.withColumn('order_revenue',sum('order_item_subtotal').over(Window.partitionBy('order_item_order_id'))). \
                              withColumn('percentage', round(col('order_item_subtotal')/col('order_revenue'), 2))
percentageoforderItemsfinal = percentageoforderItems.select('order_item_order_id', 'order_item_id','order_item_subtotal', round('order_revenue',2).alias('order_revenue'),'percentage'). \
                                                     orderBy(col('order_item_order_id'))
percentageoforderItemsfinal.coalesce(1).write.csv("/FileStore/tables/percentageoforderItemsfinal", compression="none", header ='true')
percentageoforderItemsfinal.show()

+-------------------+-------------+-------------------+-------------+----------+
order_item_order_id|order_item_id|order_item_subtotal|order_revenue|percentage|
+-------------------+-------------+-------------------+-------------+----------+
 1| 1| 299.98| 299.98| 1.0|
 2| 4| 129.99| 579.98| 0.22|
 2| 3| 250.0| 579.98| 0.43|
 2| 2| 199.99| 579.98| 0.34|
 4| 7| 150.0| 699.85| 0.21|
 4| 5| 49.98| 699.85| 0.07|
 4| 6| 299.95| 699.85| 0.43|
 4| 8| 199.92| 699.85| 0.29|
 5| 12| 299.98| 1129.86| 0.27|
 5| 13| 129.99| 1129.86| 0.12|
 5| 9| 299.98| 1129.86| 0.27|
 5| 11| 99.96| 1129.86| 0.09|
 5| 10| 299.95| 1129.86| 0.27|
 7| 15| 299.98| 579.92| 0.52|
 7| 14| 199.99| 579.92| 0.34|
 7| 16| 79.95| 579.92| 0.14|
 8| 17| 179.97| 729.84| 0.25|
 8| 20| 50.0| 729.84| 0.07|
 8| 18| 299.95| 729.84| 0.41|
 8| 19| 199.92| 729.84| 0.27|
+-------------------+-------------+-------------------+-------------+----------+
only showing top 20 rows